In [1]:
import mechanicalsoup
from bs4 import BeautifulSoup
import re
import MySQLdb
import time

In [2]:
class stock_crawler:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.browser = browser = mechanicalsoup.StatefulBrowser()
    
    def read_details():
        print('reading user data')
    def check_error(self):
        #check if there is error
        print("no error")
        
    def login(self):
        self.browser.open("https://onlinetrading.nse.co.ke/tradeweb111/")
        self.browser.select_form(nr=0)
        self.browser['txtLogin'] = self.username
        self.browser['txtPassword'] = self.password
        # Login
        self.browser.submit_selected()
        print("logging in")
        
    def crawl_ticker(self):
        self.browser.open("https://onlinetrading.nse.co.ke/tradeweb111/TWHome.aspx")
        page = self.browser.get_current_page()
        soup = BeautifulSoup(str(page), 'lxml')
        div = soup.find("div", {"class": "CSPLTickerContainer"})
        ul =  div.find("ul")
        li =  ul.find("li")
        sql = "INSERT INTO ticker (security, price, price_change) VALUES "
        for li in ul:
            combo = li.text.split('|', 1)[0]
            security_price, change = combo.split(' ')
            security = re.split('(\d+)',security_price)
            price = ''.join(security[1:4])
            #items.append([security[0] ,price , change])
            sql = sql + "('"+security[0]+"', "+ price+", "+change+"),"
        sql = sql[:-1] + ";"
        print(sql)
        
        print("ticker data collected")
        time.sleep(10)
    def crawl_market_data(self):
        self.browser.open('https://onlinetrading.nse.co.ke/tradeweb111/MarketDetails.aspx?wl=Default')
        page = self.browser.get_current_page()
        soup = BeautifulSoup(str(page), 'lxml')
        script = soup.find_all("script")
        
        jss =  str(script[22])
        str_replace_qoutes = jss.replace('\"', '\\"')
        str_minus_semi_colon = str_replace_qoutes.replace(";"," ")
        str_minus_sdarr = str_minus_semi_colon.replace("sDArr","")
        str_minus_equal = str_minus_sdarr.replace("=","")

        #everything before sdarr
        head, sep, tail = str_minus_equal.partition('(jsiScripsInMW,50) ')
        #everything after sdarr
        head2, sep2, tail2 = tail.partition('  jsiMWrefreshDelay')
        result = re.sub("[\(\[].*?[\)\]]", "", head2)
        arr = result.split("' '")
        arr[0] = arr[0].replace("\'","")
        arr[-1] = arr[-1].replace("\'","")
        
        sdarr = []
        sec = []
        count =1
        for i in range(len(arr)):
            count = count + 1
            sec.append(arr[i])
            if count == 22:
                count = 1
                sdarr.append(sec)
                sec = []
        sql = "INSERT INTO market_details (security, last_price, demand_qty, demand_price, supply_price, supply_qty, last_qty, high, low) VALUES "
        for row in sdarr:
            sub_row = [row[2], row[8], row[9], row[10], row[11], row[12]]
            high_low = self.sortlist(sub_row)
            sql = sql + "( '"+row[0]+"', "+ row[2]+", "+row[5]+", "+row[6]+", "+row[8]+", "+row[7]+", "+row[4]+", "+max(high_low)+", "+min(high_low)+"),"
        sql = sql[:-1] + ";"    
        print(sql)

        print("market data collected")
    def logout(self):
        del self
        self.browser.follow_link()
        print("logged out")
    def sortlist(self, mylist):
        newlist=[]
        for item in mylist:
            if float(item) > 0:
                newlist.append(item)

        if newlist == []:
            newlist = [str(0.00), str(0.00)]
        return newlist
    
    def db_connection(self):
        self.db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                         user="root",         # your username
                         passwd="",  # your password
                         db="nse")
        
    def db_query(self, sql):
        self.db.query(sql)
        self.db.commit()    
    def db_close(self):
        print("closing connection")
        self.db.close()
    def run(self):
        self.db_connection()
        print("logging in..")
        self.login()
        print('collecting ticker..')
        self.crawl_ticker()
        print('collecting market data..')
        self.crawl_market_data()
        self.db_close()
       

In [3]:
crawler = stock_crawler('karis','gray1236')
crawler.run()
del crawler


logging in..
logging in
collecting ticker..
INSERT INTO ticker (security, price, price_change) VALUES ('EQT', 47.75, 3.75),('KEGN', 8.65, -0.30),('KQ', 13.00, -1.20),('NSE', 21.50, 1.50);
ticker data collected
collecting market data..
INSERT INTO market_details (security, last_price, demand_qty, demand_price, supply_price, supply_qty, last_qty, high, low) VALUES ( 'XPRS', 3.75, 200, 4.65, 0.00, 0, 0, 3.75, 3.75),( 'KQ', 13.00, 1000, 12.50, 12.95, 21600, 1000, 14.20, 12.95),( 'WTK', 158.00, 6000, 145.00, 158.00, 200, 0, 158.00, 158.00),( 'UTK', 0.00, 0, 0.00, 0.00, 0, 0, 0.0, 0.0),( 'UNGA', 42.25, 1000, 37.00, 42.50, 10300, 0, 42.50, 42.25),( 'UCHM', 3.00, 56600, 2.50, 0.00, 0, 0, 3.00, 3.00),( 'TPS', 35.00, 2400, 35.00, 36.75, 100, 0, 36.75, 35.00),( 'TOTL', 29.75, 5000, 26.25, 30.00, 3300, 0, 30.00, 29.75),( 'TCL', 5.70, 1000, 4.50, 5.30, 7100, 0, 5.70, 5.30),( 'SGL', 33.00, 0, 0.00, 31.00, 1400, 0, 33.00, 31.00),( 'SCOM', 29.75, 1700, 29.50, 29.75, 1288800, 0, 29.75, 29.75),( 'SCBK',